In [1]:
import os
import io
import re
import multiprocessing

In [2]:
import neuralcoref
import spacy

print('spacy version:', spacy.__version__)
print('neuralcoref version:', neuralcoref.__version__)

spacy version: 2.1.3
neuralcoref version: 4.0.0


# Reconhecimento de entidades

In [3]:
def read_file(path):
    try:
        with io.open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        try:
            with io.open(path, 'r', encoding='latin-1') as f:
                return f.read()
        except UnicodeDecodeError:
            return "Could not decode"

In [4]:
def extract_paragraphs(folder, file):
    path = os.path.join(folder, file) # name file

    if os.path.isfile(path):
        content = read_file(path)
        print(f"Content of book ... {file}")

        paragraphs = content.split('\n\n')

        return paragraphs
    return []

In [5]:
def new_paragraphs(paragraphs):
    paragraphs_new = []
    contents = []
    chapters = []
    old_idx = 0

    for paragraph in paragraphs:
        lines = paragraph.split('\n')

        if re.match('[IVXLCDM\d]+[\.]*', lines[0].upper().strip()):
            if not contents:
                contents = [" ".join(line.lower().split()) for line in lines]
                
                if not contents:
                    print("Not found content", id_file)
                    exit()
            continue

        if paragraph.lower().strip() in contents:
            if old_idx == 0:
                old_idx = len(paragraphs_new)
            else:
                current_idx = len(paragraphs_new)
                chapter = "\n\n".join(paragraphs_new[old_idx:current_idx])
                chapters.append(chapter)
                old_idx = current_idx
            continue

        if not re.match('.*[\w]+.*', paragraph):
            continue

        paragraphs_new.append(paragraph)

        if old_idx != 0:
            chapter = "\n\n".join(paragraphs_new[old_idx:])
            chapters.append(chapter)
        
    return paragraphs_new

In [6]:
def process_paragraph(paragraph, nlp):
    paragraph = paragraph.replace('\n', ' ')
    paragraph = " ".join(paragraph.split())
    doc = nlp(paragraph)
    doc_coref = doc._.coref_resolved
    doc_coref = nlp(doc_coref)

    entities = [ent.text for ent in doc_coref.ents if ent.label_ == "PERSON"]

    return entities

def extract_entities_from_paragraphs(paragraphs_new, nlp):
    data = []
    geral_entities = []

    paragraphs_to_process = paragraphs_new  # Adjust as needed

    with multiprocessing.Pool() as pool:
        results = pool.starmap(process_paragraph, [(p, nlp) for p in paragraphs_to_process])

    for i, entities in enumerate(results):
        if entities:
            len_words = len(paragraphs_to_process[i].split())
            geral_entities.extend(entities)
            data.append((i, len_words, entities))
            # print(i, len_words, entities)

    return data, geral_entities

In [7]:
def all_entities(folder, file):     
    paragraphs = extract_paragraphs(folder, file)
    paragraphs_new = new_paragraphs(paragraphs)
    
    print('Número de parágrafos:', len(paragraphs))
    print('Número de parágrafos com personagens: ', len(paragraphs_new))     
    
    nlp = spacy.load("en_core_web_sm")
    neuralcoref.add_to_pipe(nlp)
    
    data, geral_entities = extract_entities_from_paragraphs(paragraphs_new, nlp)
    
    return data, geral_entities

# Criação de rede de co-ocorrência

In [8]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
def cooccurrence_matrix(data, entities_list):
    # Inicializa a matriz de coocorrência
    cooccurrence_matrix = np.zeros((len(entities_list), len(entities_list)), dtype=int)

    # Preenche a matriz de coocorrência
    for _, _, paragraph_entities in data:
        for i in range(len(entities_list)):
            if entities_list[i] in paragraph_entities:
                for j in range(i, len(entities_list)):
                    if entities_list[j] in paragraph_entities:
                        cooccurrence_matrix[i][j] += 1
                        if i != j:
                            cooccurrence_matrix[j][i] += 1
    return np.array(cooccurrence_matrix)

In [19]:
def edge_list(entities_list, co_matrix):
    edge_list = []

    for i in range(len(entities_list)):
        for j in range(i + 1, len(entities_list)):
            weight = co_matrix[i][j]
            if weight > 0:
                edge_list.append((entities_list[i], entities_list[j], {'weight': weight}))
    
    return edge_list

In [20]:
def plot_network_degree(G, name_list, folder):
    
    # Atribuir uma cor única a cada nó
    node_colors = range(len(name_list))

    # Identifica os 15 nós com maior grau
    degrees = dict(G.degree())
    top_degree_nodes = sorted(degrees, key=degrees.get, reverse=True)[:15]

    # Determina as arestas que conectam os nós de maior grau
    edges_of_top_degree_nodes = [(u, v) for u, v in G.edges() if u in top_degree_nodes and v in top_degree_nodes]

    # Posicionamento dos nós aleatório
    pos = nx.random_layout(G)

    plt.figure(figsize=(12, 12))

    # Desenha a rede
    nx.draw(G,
            pos=pos,
            node_color=node_colors,
            cmap=plt.get_cmap('OrRd_r'),  # mapa de cores
            node_size=35,  # Tamanho dos nós
            edge_color="#E1E5EA", # "#DBDFEA"  # Cor das arestas
            with_labels=False,  # Não mostra rótulos dos nós
            alpha= 0.7,
           ) 

    # Adiciona as arestas dos nós de maior grau em uma cor diferente
    nx.draw(G,
            pos=pos,
            node_color=node_colors,
            cmap=plt.get_cmap('OrRd_r'),  # mapa de cores
            node_size=35,  # Tamanho dos nós
            edgelist=edges_of_top_degree_nodes,
            edge_color='red',  # Cor das arestas dos nós de maior grau
            width=1.0,  # Largura das arestas
            alpha=0.2)  # Transparência das arestas


    for node in top_degree_nodes:
        x, y = pos[node]
        plt.annotate(node, (x, y), ha='center', va='center', fontsize=10,
                     # fontweight='heavy',
                     color='#461959', #B80D57 #99235C
                     backgroundcolor='white',
                     bbox=dict(boxstyle='round,pad=0.2', edgecolor='#D3CEDF', facecolor='#F9F5F6'))

    plt.title("Rede de co-ocorrência livro: " +  str(i+1), fontsize=16)
    plt.axis('off')
    plt.savefig("graphs/Graph_" + folder + "_" +  str(i+1) + '.png', dpi=300, bbox_inches='tight')
    plt.show()

# Medidas de centralidade

In [12]:
import pandas as pd

def medidas(G):
    
    K = dict(nx.degree_centrality(G)) # Centralidade de grau
    CC = dict(nx.clustering(G)) # Clustering coeficient
    CLC = dict(nx.closeness_centrality(G)) # Closeness centrality
    B = dict(nx.betweenness_centrality(G, endpoints=True)) # Betweenness centrality
    EC = dict(nx.eigenvector_centrality(G)) # Eigenvetor centrality
    PR = dict(nx.pagerank(G, alpha=0.85)) # PageRank
    
    
    df = pd.DataFrame({'K':list(K.values()),
                       'CC':list(CC.values()),
                       'CLC':list(CLC.values()),
                       'B':list(B.values()),
                       'EC':list(EC.values()),
                       'PR':list(PR.values())})
    display(df)
    
    # Vetor com valores médios das medidas de rede de co-ocorrência
    medidas = list(df.mean())
    
    return medidas

In [21]:
def Medidas(folder):
    
    files_ordered = sorted(os.listdir(folder), key=lambda name:int(name.split('_')[0]))
    
    M = list()
    for i in range(len(files_ordered)):

        file = files_ordered[i]
        print('\n')

        data, geral_entities = all_entities(folder, file)

        # Lista das entidades (personagens)
        entities_list = list(set(geral_entities))

        # Matriz de coocorrência
        co_matrix = cooccurrence_matrix(data, entities_list)
        print("Matriz de Co-ocorrência", co_matrix.shape)

        # Lista de arestas
        edgelist = edge_list(entities_list, co_matrix)

        # Rede
        G = nx.Graph()
        G.add_nodes_from(entities_list)
        G.add_edges_from(edgelist)

        print('Personagens (nodes)', G.number_of_nodes())
        print('Ocorrência (edges)', G.number_of_edges())
        
        # Visualização de rede - 15 personagens com maior grau
        # plot_network_degree(G, entities_list, folder, file)

        medidas_rede = medidas(G)
        print('Valores médios das medidas de rede:\n', medidas_rede)

        M.append(medidas_rede) 
        
    return M

In [22]:
medidas_other = Medidas(folder="other")



Content of book ... 1_InTheLandOfCaveAndCliffDwellers_FrederickSchwatka_cleared.txt
Número de parágrafos: 301
Número de parágrafos com personagens:  191
Matriz de Co-ocorrência (74, 74)
Personagens (nodes) 74
Ocorrência (edges) 78


,K,CC,CLC,B,EC,PR
0,0.041096,1.000000,0.041096,0.001111,1.914859e-11,0.017036
1,0.000000,0.000000,0.000000,0.000000,2.416892e-40,0.002555
2,0.082192,0.666667,0.110241,0.009256,3.488031e-01,0.022500
3,0.013699,0.000000,0.082681,0.004813,8.465884e-02,0.005644
4,0.013699,0.000000,0.091532,0.007775,6.514187e-06,0.006083
...,...,...,...,...,...,...
69,0.027397,1.000000,0.027397,0.000740,1.927869e-17,0.017036
70,0.082192,0.333333,0.131328,0.017216,2.823667e-05,0.037351
71,0.054795,1.000000,0.096461,0.004813,1.873479e-01,0.016054
72,0.027397,1.000000,0.027397,0.000740,1.927869e-17,0.017036


Valores médios das medidas de rede:
 [0.028878193261754906, 0.40067567567567564, 0.059625055724921355, 0.00557351131212664, 0.0443319050197565, 0.013513513513513514]


Content of book ... 2_JudeTheObscure_ThomasHardy_cleared.txt
Número de parágrafos: 3689
Número de parágrafos com personagens:  3375
Matriz de Co-ocorrência (240, 240)
Personagens (nodes) 240
Ocorrência (edges) 326


,K,CC,CLC,B,EC,PR
0,0.117155,0.132275,0.272209,0.042035,2.504565e-01,0.050249
1,0.016736,1.000000,0.132973,0.004672,1.207040e-03,0.004569
2,0.008368,1.000000,0.188295,0.004672,2.575550e-02,0.001755
3,0.000000,0.000000,0.000000,0.000000,7.907440e-23,0.000790
4,0.000000,0.000000,0.000000,0.000000,7.907440e-23,0.000790
...,...,...,...,...,...,...
235,0.008368,1.000000,0.224939,0.004672,5.691187e-02,0.001629
236,0.004184,0.000000,0.004184,0.000035,4.145776e-17,0.005267
237,0.000000,0.000000,0.000000,0.000000,7.907440e-23,0.000790
238,0.025105,0.600000,0.171922,0.015830,6.076626e-03,0.007078


Valores médios das medidas de rede:
 [0.011366806136680611, 0.38194760572684516, 0.11725293584819303, 0.005031090190609018, 0.03201330925314385, 0.0041666666666666675]


Content of book ... 3_TheGoldenAge_KennethGrahame_cleared.txt
Número de parágrafos: 522
Número de parágrafos com personagens:  425
Matriz de Co-ocorrência (121, 121)
Personagens (nodes) 121
Ocorrência (edges) 213


...

,K,CC,CLC,B,EC,PR
0,0.005236,0.0,0.015956,0.000436,2.627696e-10,0.003673
1,0.020942,1.0,0.036813,0.000818,8.540843e-06,0.007782
2,0.005236,0.0,0.005236,0.000055,3.497088e-17,0.007008
3,0.015707,1.0,0.044677,0.000873,7.789242e-02,0.005084
4,0.005236,0.0,0.005236,0.000055,3.497088e-17,0.007008
...,...,...,...,...,...,...
187,0.005236,0.0,0.018057,0.000709,8.143226e-11,0.003623
188,0.015707,1.0,0.044677,0.000873,7.789242e-02,0.005084
189,0.000000,0.0,0.000000,0.000000,5.211068e-25,0.001051
190,0.010471,1.0,0.010471,0.000109,1.324583e-12,0.007008


Valores médios das medidas de rede:
 [0.009598603839441536, 0.3546502976190476, 0.01479205847390626, 0.0004158486038394415, 0.01735424802836948, 0.005208333333333333]


Content of book ... 25_EvelynInnes_GeorgeMoore_cleared.txt
Número de parágrafos: 2606
Número de parágrafos com personagens:  2393
Matriz de Co-ocorrência (325, 325)
Personagens (nodes) 325
Ocorrência (edges) 924


,K,CC,CLC,B,EC,PR
0,0.012346,1.000000,0.340345,0.005147,3.417087e-02,0.001644
1,0.003086,0.000000,0.259645,0.005147,3.248691e-03,0.000850
2,0.006173,1.000000,0.362672,0.005147,3.724889e-02,0.001035
3,0.012346,1.000000,0.280185,0.005147,1.146688e-02,0.001743
4,0.049383,0.233333,0.374661,0.020370,5.921976e-02,0.006988
...,...,...,...,...,...,...
320,0.006173,1.000000,0.345007,0.005147,2.864784e-02,0.001027
321,0.000000,0.000000,0.000000,0.000000,9.442153e-26,0.000522
322,0.006173,1.000000,0.294759,0.005147,1.300717e-02,0.001334
323,0.006173,1.000000,0.255259,0.005147,2.127929e-03,0.001722


Valores médios das medidas de rede:
 [0.01754985754985755, 0.5501494088893214, 0.27818598805350414, 0.007696632332529768, 0.033613928157807706, 0.003076923076923076]


Content of book ... 26_TheGoldenCanyon_GAHenty_cleared.txt
Número de parágrafos: 1288
Número de parágrafos com personagens:  1114
Matriz de Co-ocorrência (64, 64)
Personagens (nodes) 64
Ocorrência (edges) 86


PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [ ]:
medidas_success = Medidas(folder="success")

# Classificação

In [ ]:
# other - Others (1895 - 1923)
dfo = pd.read_csv('catalog_v0.xlsx - other.csv')
dfo['Classe'] = 0
dfo['Medidas'] = medidas_other
dfo.head()

In [ ]:
# success- Publishers Weekly’s Bestselling Novels
dfs = pd.read_csv('catalog_v0.xlsx - success.csv')
dfs['Classe'] = 1
dfs['Medidas'] = medidas_success
dfs.head()

In [ ]:
concat = pd.concat([dfs, dfo], axis=0)
concat = concat.sample(frac=1).reset_index(drop=True)
display(concat)

In [ ]:
for i in range(len(concat)):
    name = ''.join(concat["Title"][i].split())
    index = concat["Index"][i]
    namesearch = str(index) + '_' + str(name)
    print(namesearch)

In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter